In [32]:
import tensorflow as tf 
import cv2
from tensorflow.examples.tutorials.mnist import input_data
from scipy import ndimage
import numpy as np
import math

def getBestShift(img):
    cy,cx = ndimage.measurements.center_of_mass(img)

    rows,cols = img.shape
    shiftx = np.round(cols/2.0-cx).astype(int)
    shifty = np.round(rows/2.0-cy).astype(int)

    return shiftx,shifty
    
def getBestShift(img):
    cy,cx = ndimage.measurements.center_of_mass(img)

    rows,cols = img.shape
    shiftx = np.round(cols/2.0-cx).astype(int)
    shifty = np.round(rows/2.0-cy).astype(int)

    return shiftx,shifty

def shift(img,sx,sy):
    rows,cols = img.shape
    M = np.float32([[1,0,sx],[0,1,sy]])
    shifted = cv2.warpAffine(img,M,(cols,rows))
    return shifted

# create a MNIST_data folder with the MNIST dataset if necessary   
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

"""
a placeholder for our image data:
None stands for an unspecified number of images
784 = 28*28 pixel
"""
x = tf.placeholder("float", [None, 784])

# we need our weights for our neural net
W = tf.Variable(tf.zeros([784,10]))
# and the biases
b = tf.Variable(tf.zeros([10]))

"""
softmax provides a probability based output
we need to multiply the image values x and the weights
and add the biases
(the normal procedure, explained in previous articles)
"""
y = tf.nn.softmax(tf.matmul(x,W) + b)

"""
y_ will be filled with the real values
which we want to train (digits 0-9)
for an undefined number of images
"""
y_ = tf.placeholder("float", [None,10])

"""
we use the cross_entropy function
which we want to minimize to improve our model
"""
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

"""
use a learning rate of 0.01
to minimize the cross_entropy error
"""
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

# initialize all variables
init = tf.initialize_all_variables()

# create a session
sess = tf.Session()
sess.run(init)

# use 1000 batches with a size of 100 each to train our net
for i in range(10000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  # run the train_step function with the given image values (x) and the real output (y_)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

sess.run(train_step,feed_dict={x:newTrain, y_:6})

"""
Let's get the accuracy of our model:
our model is correct if the index with the highest y value
is the same as in the real digit vector
The mean of the correct_prediction gives us the accuracy.
We need to run the accuracy function
with our test set (mnist.test)
We use the keys "images" and "labels" for x and y_
"""
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print( "\nTraining accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

################ END OF TRAINING ###############################

# create an array where we can store our 4 pictures
images = np.zeros((1,784))
# and the correct values
correct_vals = np.zeros((1,10))

# read the image
origImage = cv2.imread("form4.png", cv2.IMREAD_GRAYSCALE)

for R in range(10):
    print("\nRow ",R, ":",end=" ")
    for C in range(20):
        # Crop the image to a single digit (should be in a loop to scan)
        # X and Y are counter-intuitive (switched row vs col)
        ##### Example 0:  Number 0
        offsetx = 110+(R*48) 
        offsety = 131+(C*52) 
        gray = origImage[offsetx:offsetx+44, offsety:offsety+48]    

        ##### Example 1:  Number 7 -- correct
        #offsetx = 301                                    
        #offsety = 911 
        #gray = gray[offsetx:offsetx+50, offsety:offsety+50]    

        ##### Example 2:  Number 2 -- correct
        #offsetx = 351                                    
        #offsety = 811 
        #gray = gray[offsetx:offsetx+50, offsety:offsety+50]    

        ##### Example 3:  Number 3 -- correct
        #offsetx = 256                                    
        #offsety = 653
        #gray = gray[offsetx:offsetx+50, offsety:offsety+50]    

        ##### Example 4:  Number 7 -- correct
        #offsetx = 208                                    
        #offsety = 963
        #gray = gray[offsetx:offsetx+50, offsety:offsety+50]    

        ##### Example 5:  Number 6 -- correct
        #offsetx = 348                                    
        #offsety = 1065
        #gray = gray[offsetx:offsetx+50, offsety:offsety+50]    


        ##################### Pre-process the image ##########################

        # resize the images and invert it (black background)
        gray = cv2.resize(255-gray, (28, 28))

        (thresh, gray) = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

        while np.sum(gray[0]) == 0:
            gray = gray[1:]

        while np.sum(gray[:,0]) == 0:
            gray = np.delete(gray,0,1)

        while np.sum(gray[-1]) == 0:
            gray = gray[:-1]

        while np.sum(gray[:,-1]) == 0:
            gray = np.delete(gray,-1,1)

        rows,cols = gray.shape

        if rows > cols:
            factor = 20.0/rows
            rows = 20
            cols = int(round(cols*factor))
            gray = cv2.resize(gray, (cols,rows))
        else:
            factor = 20.0/cols
            cols = 20
            rows = int(round(rows*factor))
            gray = cv2.resize(gray, (cols, rows))

        colsPadding = (int(math.ceil((28-cols)/2.0)),int(math.floor((28-cols)/2.0)))
        rowsPadding = (int(math.ceil((28-rows)/2.0)),int(math.floor((28-rows)/2.0)))
        gray = np.lib.pad(gray,(rowsPadding,colsPadding),'constant')

        shiftx,shifty = getBestShift(gray)
        shifted = shift(gray,shiftx,shifty)
        gray = shifted

        # save the processed images
        cv2.imwrite("proc_image"+str(R)+str(C)+".png", gray)

        ################ DONE Processing image ################

        #"""
        #all images in the training set have an range from 0-1
        #and not from 0-255 so we divide our flatten images
        #(a one dimensional vector with our 784 pixels)
        #to use the same 0-1 based range
        #"""
        flatten = gray.flatten() / 255.0
        #"""
        #we need to store the flatten image and generate
        #the correct_vals array
        #correct_val for the first digit (9) would be
        #[0,0,0,0,0,0,0,0,0,1]
        #"""
        i=0
        images[i] = flatten
        correct_val = np.zeros((10))
        #correct_val[2] = 1
        correct_vals[i] = correct_val
        i += 1

        #"""
        #the prediction will be an array with four values,
        #which show the predicted number
        #"""
        prediction = tf.argmax(y,1)
        #"""
        #we want to run the prediction and the accuracy function
        #using our generated arrays (images and correct_vals)
        #"""
        #print("Top predicted value: ", sess.run(prediction, feed_dict={x: images, y_: correct_vals}))
        print(sess.run(prediction, feed_dict={x: images, y_: correct_vals}), end="")
        #print(sess.run(accuracy, feed_dict={x: images, y_: correct_vals}))

        pred_val = tf.argmax(y, 1)
        pred_softmax = tf.nn.softmax(y)
        pred_top_5 = tf.nn.top_k(pred_softmax, k=5, sorted=True)
        final_value_top_5, result_top_5 = sess.run([pred_val, pred_top_5], feed_dict={x: images, y_: correct_vals})

        #print("Top Five Values: ", result_top_5[1])
        #print("Top Five Weights: ", result_top_5[0])
        #print("Top Weight: ", result_top_5[0][0][0])
        #print("Next Weight: ", result_top_5[0][0][1])

        if (result_top_5[0][0][1] >= (0.8*result_top_5[0][0][0])): 
            print("*", end="")
            #print("Warning:  Top 2 values in close range!!")
            #print("Top Five Values: ", result_top_5[1])

            

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz

Training accuracy: 0.9231

Row  0 : [0][1][1][0][1][1][0][1][1][0][5][4][3][3][5][5][7][8][8][9]
Row  1 : [1][1][1][1][0][0][0][1][0][1][3][3][5][4][3][7][6][8][3]*[7]
Row  2 : [5][1][1][1][1][1][1][8][0][1][3][2][3][2][4][7]*[7][8][5]*[5]*
Row  3 : [1][1][0][1][1][0][0][0][1][0][3][3][4][4][5][8][8][7][5][8]
Row  4 : [0][1][0][1][0][1][0][1][5][3]*[4][5][3][4][5]*[7][5][8][6][9]
Row  5 : [1][1][0][1][0][1][1][1][1][0][5][5][8]*[3][4][7][5][7][6][1]
Row  6 : [1][0][1][0][1][0][1][0][1][0][8][1][1][4][2][8][8][6][8][7]
Row  7 : [1][0][0][1][1][1][0][0][0][1][2][0][7][4][3][8][7][6][1][8]*
Row  8 : [1][0][1][1][1][1][1][0][0][1][7]*[8][2][5][3][8][7][8][7][6]
Row  9 : [1][1][1][1][5]*[0][0][0][1][1][4][4]*[5][5][3][6][1][5][3][7]